In [2]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
from pathlib import Path

import pandas as pd
from tokenizers import ByteLevelBPETokenizer
from src.config import config

bodies = pd.read_parquet(config.data.news.stripped, columns=["parsed_body"])

In [12]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train_from_iterator(bodies.parsed_body.values, 
                              vocab_size=30000, 
                              min_frequency=10, 
                              special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
                              ])

In [18]:
# Save files to disk
tokenizer.save_model("data/models/newstokenizer")

['data/models/newstokenizer/vocab.json',
 'data/models/newstokenizer/merges.txt']

In [32]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "data/models/newstokenizer/vocab.json",
    "data/models/newstokenizer/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

x = tokenizer.encode("the company is showing strong profits this second quarter. weak profits last quarter were announced though.")
print(
    x
)
# Encoding(num_tokens=7, ...)
# tokens: ['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

Encoding(num_tokens=20, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [33]:
x.ids

[0,
 4925,
 321,
 335,
 2486,
 760,
 2766,
 402,
 901,
 409,
 18,
 2073,
 2766,
 694,
 409,
 639,
 1080,
 2046,
 18,
 2]

In [34]:
for id in x.ids:
    print(tokenizer.decode([id]))

<s>
the
 company
 is
 showing
 strong
 profits
 this
 second
 quarter
.
 weak
 profits
 last
 quarter
 were
 announced
 though
.
</s>
